In [2]:
!pip install numpy pandas matplotlib scikit-learn tensorflow keras opencv-python

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

2025-04-17 11:20:14.695673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 11:20:14.708985: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 11:20:14.819419: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 11:20:14.947504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744899615.099760    8260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744899615.12

In [5]:
CAMINHO_IMAGENS = "/home/gustavson-barros/Área de trabalho/ProjetoDeExtensao/dataset/archive/ham10000_images_part_1"

CAMINHO_CSV = "/home/gustavson-barros/Área de trabalho/ProjetoDeExtensao/dataset/archive/HAM10000_metadata.csv"

df = pd.read_csv(CAMINHO_CSV)
df = df[:100]  # apenas 100 para testar

IMG_SIZE = 128



In [6]:
imagens = []
labels = []

for index, row in df.iterrows():
    nome_arquivo = row["image_id"] + ".jpg"
    classe = row["dx"]  # tipo de lesão

    caminho_completo = os.path.join(CAMINHO_IMAGENS, nome_arquivo)
    
    # Verifica se a imagem existe
    if os.path.exists(caminho_completo):
        img = cv2.imread(caminho_completo)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        imagens.append(img)
        labels.append(classe)

# Converte listas para arrays NumPy
X = np.array(imagens) / 255.0  # Normalização
y = np.array(labels)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categ = to_categorical(y_encoded)

In [7]:
#treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y_categ, test_size=0.2, random_state=42)

In [8]:
# Criação da CNN simples
modelo = Sequential()
modelo.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
modelo.add(MaxPooling2D((2, 2)))
modelo.add(Flatten())
modelo.add(Dense(64, activation='relu'))
modelo.add(Dense(y_categ.shape[1], activation='softmax')) 

/home/gustavson-barros/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-17 11:32:14.332388: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Treina o modelo
modelo.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 809ms/step - accuracy: 0.2278 - loss: 0.7405 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 348ms/step - accuracy: 0.9751 - loss: 0.7482 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step - accuracy: 0.9855 - loss: 0.5629 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step - accuracy: 0.9855 - loss: 0.5511 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.9751 - loss: 0.9217 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [11]:
def prever_imagem(caminho_img):
    img = cv2.imread(caminho_img)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.expand_dims(img / 255.0, axis=0)
    pred = modelo.predict(img)
    classe = le.inverse_transform([np.argmax(pred)])
    return classe[0]

print(prever_imagem("/home/gustavson-barros/Área de trabalho/ProjetoDeExtensao/dataset/archive/ham10000_images_part_1/ISIC_0024306.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
bkl
